Cyna Shirazinejad, 1/13/22

# outline of notebook 7:

* load data from cell lines:
* * AP2-tagRFP-T, tagGFP2-DNM2, ARPC3-HaloTag 
* * AP2-tagRFP-T, tagGFP2-DNM2, N-WASP-HaloTag 
* extract features from tracks
* * use existing feature scaler, decomposition axes, and mixture model to predict the identity of each new event
* merge the new data with existing tracks, features, and model cluster identities

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import sys
analysis_metadata = np.load('analysis_metadata.npy', allow_pickle=True)
sys.path.append(analysis_metadata.item().get('path_notebook')+'/cmeAnalysisPostProcessingSimplified') # add custom Python scripts to the local path
import import_tracks
import display_tracks
import detect_peaks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
analysis_metadata

array({'path_outputs': '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_20220211', 'path_notebook': '/Users/cynashirazinejad/Documents/GitHub/Jin_Shirazinejad_et_al_branched_actin_manuscript/analysis/simplified_workflow_20220211', 'feature_units': ['seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'pixels', 'pixels', 'pixels', 'pixels', 'pixels', 'seconds', 'seconds', 'seconds', 'seconds', 'seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'unitless', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence**2', 'a.u. fluorescence**2', 'unitless', 'unitless', 'unitless', 'unitless', 'counts', 'counts', 'unitless', 'unitless', 'unitless'], 'possible_track_features': ['lifetime', 'max_int_ch0', 'max_int_ch1', 'dist_traveled_ch0', 'dist_traveled_ch1', 'max_dist_between_ch0-ch1', 'md_ch0', 'md_ch1', 'time_to_peak_ch0', 'time_to_peak_ch1', 'time_after_peak_ch0', 'time_after_peak_ch1', 'time

In [5]:
display_tracks.upload_tracks_and_metadata?

Signature:
display_tracks.upload_tracks_and_metadata(
    path_tracks,
    path_outputs,
    analysis_metadata,
    track_categories,
    identifier_string,
    features,
    labels,
    dataframe_name,
    track_name,
    experiment_number_adjustment=0,
)
Docstring:
Format tracks contained in folders into a dataframe of extracted physical features 
as well as accompanying experimental metadata.

Args:
    analysis_metadata (dictionary): contains path to the folder containing the enclosed tracking files and output files
    track_categories (list): a list of integers for the cmeAnalysis track categories to keep for further analysis
    identifier_string (string): a label within all tracking head folders that uniquely identify relevant content
    features (list): a list with string elements containing the features to be extracted from each track
    labels (list): a list of string elements describing each features' designation in the dataframe output
    experiment_number_adjustment (i

In [6]:
possible_track_features = analysis_metadata.item().get('possible_track_features')
possible_track_features_labels = analysis_metadata.item().get('possible_track_features_labels')

In [7]:
analysis_metadata

array({'path_outputs': '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_20220211', 'path_notebook': '/Users/cynashirazinejad/Documents/GitHub/Jin_Shirazinejad_et_al_branched_actin_manuscript/analysis/simplified_workflow_20220211', 'feature_units': ['seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'pixels', 'pixels', 'pixels', 'pixels', 'pixels', 'seconds', 'seconds', 'seconds', 'seconds', 'seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'unitless', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence**2', 'a.u. fluorescence**2', 'unitless', 'unitless', 'unitless', 'unitless', 'counts', 'counts', 'unitless', 'unitless', 'unitless'], 'possible_track_features': ['lifetime', 'max_int_ch0', 'max_int_ch1', 'dist_traveled_ch0', 'dist_traveled_ch1', 'max_dist_between_ch0-ch1', 'md_ch0', 'md_ch1', 'time_to_peak_ch0', 'time_to_peak_ch1', 'time_after_peak_ch0', 'time_after_peak_ch1', 'time

# 3 color cell line (including arpc3 tag in addition to ap2/dnm2): upload tracks, extract features, find clusters and ccp's

In [8]:
path_ada_tracks = '/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/tracking_data/ADA_cellline_analysis_formatted_temp/'

In [11]:
ada_wildtype_group = {}
ada_wildtype_group['path'] = path_ada_tracks
ada_wildtype_group['df'] = 'df_ada_wildtype_merged_features'
ada_wildtype_group['tracks'] = 'merged_ada_wildtype_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_wildtype'] = ada_wildtype_group
np.save('analysis_metadata', analysis_metadata)

In [13]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_wildtype',
                                         [1],
                                         ['Cell'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
8_200804_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell001_1s
9_200804_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell002_1s
10_200804_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell003_1s
11_200804_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell008_1s
12_200804_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell009_1s
13_200819_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell001_1s
14_200819_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell002_1s
15_200819_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell003_1s
16_200722_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell001_1s
17_200722_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell002_1s
18_200722_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell003_1s
19_200722_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell004_1s
20_200722_ap2-dnm2-arpc3_ap2-dnm2_wildtype_no-treatment_Cell005_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extractio

In [18]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_wildtype')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,197.0,1807.957032,1229.909514,10.128880,11.523833,3.863201,0.515501,0.713180,103.0,143.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,10.846092,-6.433661,0
1,195.0,1104.917675,1558.360759,24.093699,22.910047,3.158071,0.301681,0.534243,64.0,120.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,13.566803,-4.639073,0
2,194.0,1103.108158,1827.655903,29.493246,29.710737,4.548993,0.390344,0.644309,149.0,196.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,10.672425,-6.672535,0
3,185.0,669.325535,311.324581,36.450172,33.370516,3.670640,0.307503,0.596749,153.0,126.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,8.964181,-5.059152,0
4,184.0,968.791028,571.785312,13.916530,10.736709,2.575172,0.410520,0.663012,119.0,120.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,9.470102,-5.557483,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95120,2.0,71.707995,35.951988,5.059682,5.014421,2.181787,0.766658,1.257316,5.0,4.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,-6.021229,0.702480,1
95121,2.0,25.338391,38.053364,2.261592,2.067468,4.632292,1.427061,1.486410,6.0,2.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,4.481692,5.444504,2
95122,2.0,207.539790,166.846303,2.391855,2.634261,4.270496,1.536834,1.356576,5.0,0.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,3.839489,7.944841,2
95123,2.0,16.610995,36.530651,3.546351,2.761683,3.700703,1.630578,1.907946,5.0,9.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,2.769987,4.693659,2


In [23]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_wildtype')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,197.0,1807.957032,1229.909514,10.128880,11.523833,3.863201,0.515501,0.713180,103.0,143.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,10.846092,-6.433661,0,0.0
1,195.0,1104.917675,1558.360759,24.093699,22.910047,3.158071,0.301681,0.534243,64.0,120.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,13.566803,-4.639073,0,0.0
2,194.0,1103.108158,1827.655903,29.493246,29.710737,4.548993,0.390344,0.644309,149.0,196.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,10.672425,-6.672535,0,0.0
3,185.0,669.325535,311.324581,36.450172,33.370516,3.670640,0.307503,0.596749,153.0,126.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,8.964181,-5.059152,0,0.0
4,184.0,968.791028,571.785312,13.916530,10.736709,2.575172,0.410520,0.663012,119.0,120.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200804,9.470102,-5.557483,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95120,2.0,71.707995,35.951988,5.059682,5.014421,2.181787,0.766658,1.257316,5.0,4.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,-6.021229,0.702480,1,0.0
95121,2.0,25.338391,38.053364,2.261592,2.067468,4.632292,1.427061,1.486410,6.0,2.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,4.481692,5.444504,2,0.0
95122,2.0,207.539790,166.846303,2.391855,2.634261,4.270496,1.536834,1.356576,5.0,0.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,3.839489,7.944841,2,0.0
95123,2.0,16.610995,36.530651,3.546351,2.761683,3.700703,1.630578,1.907946,5.0,9.0,...,ap2-dnm2-arpc3,ap2-dnm2,wildtype,no-treatment,1s,200722,2.769987,4.693659,2,0.0


<!-- # upload hypotonic shock data -->

In [1]:
# path_hypotonic='/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/hypotonic_tracking_data_ap2dnm2'

In [2]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_hypotonic,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_shock_merged_features',
#                                        'merged_all_valid_ada_shock_tracks')

In [3]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_shock_merged_features')

In [4]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_shock_merged_features',
#                                                     'merged_all_valid_ada_shock_tracks')

<!-- # separately upload shock data, only keeping movies before treatment -->

In [5]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_hypotonic,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell', 'before'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_shock_before_merged_features',
#                                        'merged_all_valid_ada_before_shock_tracks')

In [6]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_shock_before_merged_features')

In [7]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_shock_before_merged_features',
#                                                     'merged_all_valid_ada_before_shock_tracks')

<!-- # separately upload shock data, only keeping movies after treatment -->

In [8]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_hypotonic,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell', 'after'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_shock_after_merged_features',
#                                        'merged_all_valid_ada_after_shock_tracks')

In [9]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_shock_after_merged_features')

In [10]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_shock_after_merged_features',
#                                                     'merged_all_valid_ada_after_shock_tracks')

<!-- # upload ck666 experiment -->

In [11]:
# path_drugs='/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/ck666_tracking_data_ap2dnm2'

In [12]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_drugs,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_drug_merged_features',
#                                        'merged_all_valid_ada_drug_tracks')

In [13]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_drug_merged_features')

In [14]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_drug_merged_features',
#                                                     'merged_all_valid_ada_drug_tracks')

<!-- # hypotonic data, second imaging date 211208, before shock -->

In [15]:
# path_hypotonic_211208='/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/hypotonic_tracking_data_211208_ap2dnm2'

In [16]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_hypotonic_211208,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell', 'before'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_shock_before_repeat_2_merged_features',
#                                        'merged_all_valid_ada_shock_before_repeat_2tracks')

In [17]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_shock_before_repeat_2_merged_features')

In [18]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_shock_before_repeat_2_merged_features',
#                                                     'merged_all_valid_ada_shock_before_repeat_2tracks')

<!-- # hypotonic data, second imaging date 211208, after shock -->

In [19]:
# import_tracks.upload_tracks_and_metadata(
#                                        path_hypotonic,  
#                                        analysis_metadata.item().get('path_outputs'),
#                                        analysis_metadata,
#                                        [1],
#                                        ['Cell', 'after'],
#                                        possible_track_features,
#                                        possible_track_features_labels,
#                                        'df_ada_shock_after_repeat_2_merged_features',
#                                        'merged_all_valid_ada_shock_after_repeat_2tracks')

In [21]:
# display_tracks.cluster_with_existing_model(analysis_metadata.item().get('path_outputs'),
#                             'df_ada_shock_after_repeat_2_merged_features')

In [22]:
# detect_peaks.find_single_peaks_using_existing_model(analysis_metadata.item().get('path_outputs'),
#                                                     'df_ada_shock_after_repeat_2_merged_features',
#                                                     'merged_all_valid_ada_shock_after_repeat_2tracks')

# upload ck666 experiment, split up by no-treatment, dmso, ck666

In [24]:
path_drugs='/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/ck666_tracking_data_ap2dnm2'

# no treatment

In [25]:
ada_ck666_nt_group = {}
ada_ck666_nt_group['path'] = path_drugs
ada_ck666_nt_group['df'] = 'df_ada_ck666_nt_merged_features'
ada_ck666_nt_group['tracks'] = 'merged_ada_ck666_nt_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_ck666_nt'] = ada_ck666_nt_group
np.save('analysis_metadata', analysis_metadata)

In [27]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_ck666_nt',
                                         [1],
                                         ['Cell', 'no-treatment'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
43_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell1_1s
44_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell2_1s
45_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell3_1s
46_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell4_1s
47_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell5_1s
48_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell6_1s
55_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell1_1s
56_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell2_1s
57_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell3_1s
58_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell4_1s
59_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell5_1s
60_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_no-treatment_Cell6_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [28]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_ck666_nt')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,217.0,1057.457912,1321.398890,17.012143,20.431176,3.296046,0.606010,0.893012,195.0,113.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,10.783667,-4.629167,0
1,210.0,1089.755190,663.638208,9.370388,4.053324,3.503022,0.275850,0.537898,102.0,182.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,10.256096,-6.237037,0
2,195.0,994.892759,481.163790,18.733539,20.331828,4.099422,0.382590,0.945931,145.0,195.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.448535,-7.158530,0
3,190.0,1561.773988,280.161725,2.969870,3.309675,3.424889,0.322540,0.864667,147.0,176.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.368589,-6.344652,0
4,189.0,1455.527836,295.281261,15.274023,15.120717,2.679219,0.351074,0.731211,124.0,141.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.991373,-6.233673,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75028,2.0,17.119725,12.390443,4.094892,6.569203,4.319224,0.662034,2.861872,5.0,5.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.811924,2.423694,3
75029,2.0,17.688742,7.296340,1.325288,3.700324,4.557518,0.962479,0.584566,6.0,3.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-7.064082,2.781733,3
75030,2.0,92.093393,19.699511,2.416176,3.646244,4.743591,0.659454,1.760703,5.0,10.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.310779,-0.545585,1
75031,2.0,16.037964,11.592347,5.632952,4.961901,3.471712,1.730392,1.714420,5.0,3.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.792258,2.582575,3


In [29]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_ck666_nt')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,217.0,1057.457912,1321.398890,17.012143,20.431176,3.296046,0.606010,0.893012,195.0,113.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,10.783667,-4.629167,0,0.0
1,210.0,1089.755190,663.638208,9.370388,4.053324,3.503022,0.275850,0.537898,102.0,182.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,10.256096,-6.237037,0,0.0
2,195.0,994.892759,481.163790,18.733539,20.331828,4.099422,0.382590,0.945931,145.0,195.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.448535,-7.158530,0,1.0
3,190.0,1561.773988,280.161725,2.969870,3.309675,3.424889,0.322540,0.864667,147.0,176.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.368589,-6.344652,0,0.0
4,189.0,1455.527836,295.281261,15.274023,15.120717,2.679219,0.351074,0.731211,124.0,141.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211214,9.991373,-6.233673,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75028,2.0,17.119725,12.390443,4.094892,6.569203,4.319224,0.662034,2.861872,5.0,5.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.811924,2.423694,3,0.0
75029,2.0,17.688742,7.296340,1.325288,3.700324,4.557518,0.962479,0.584566,6.0,3.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-7.064082,2.781733,3,0.0
75030,2.0,92.093393,19.699511,2.416176,3.646244,4.743591,0.659454,1.760703,5.0,10.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.310779,-0.545585,1,0.0
75031,2.0,16.037964,11.592347,5.632952,4.961901,3.471712,1.730392,1.714420,5.0,3.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,no-treatment,1s,211221,-6.792258,2.582575,3,0.0


# dmso

In [30]:
ada_ck666_dmso_group = {}
ada_ck666_dmso_group['path'] = path_drugs
ada_ck666_dmso_group['df'] = 'df_ada_ck666_dmso_merged_features'
ada_ck666_dmso_group['tracks'] = 'merged_ada_ck666_dmso_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_ck666_dmso'] = ada_ck666_dmso_group
np.save('analysis_metadata', analysis_metadata)

In [31]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_ck666_dmso',
                                         [1],
                                         ['Cell', 'dmso'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
34_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell1_1s
35_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell2_1s
36_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell3_1s
40_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell1_1s
41_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell2_1s
42_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell3_1s
52_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell1_1s
53_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell2_1s
54_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_dmso_Cell3_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [32]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_ck666_dmso')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,217.0,1946.020370,882.958667,6.821258,6.874475,2.824857,0.220082,0.472734,161.0,144.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,10.312214,-5.935866,0
1,210.0,779.754663,384.319760,19.708604,18.811859,5.067431,0.475209,0.789540,108.0,66.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,9.763165,-4.163253,0
2,203.0,1558.081794,440.186028,19.898377,24.494291,5.832117,0.333387,0.504222,149.0,105.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,12.027867,-4.797089,0
3,202.0,750.921810,531.726206,1.183281,7.025863,6.972678,0.434480,1.143586,161.0,169.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,9.468642,-5.427308,0
4,201.0,424.333094,156.669725,16.976344,14.241267,6.072868,0.339266,0.712789,105.0,151.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,8.638670,-5.537292,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55959,2.0,14.176969,14.112522,5.592409,4.014829,4.165132,1.519961,2.022739,5.0,7.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-6.929020,2.694182,3
55960,2.0,17.252419,8.129460,5.073412,4.869400,3.681744,1.881460,1.838313,6.0,7.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-7.241755,2.280913,3
55961,2.0,19.016496,2.305615,8.628081,5.526589,5.818022,2.927572,1.447567,6.0,3.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-7.692024,3.093005,3
55962,2.0,29.013724,14.713282,4.932474,3.433771,3.853484,1.959405,1.111740,5.0,2.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-6.790147,2.375580,3


In [33]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_ck666_dmso')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,217.0,1946.020370,882.958667,6.821258,6.874475,2.824857,0.220082,0.472734,161.0,144.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,10.312214,-5.935866,0,1.0
1,210.0,779.754663,384.319760,19.708604,18.811859,5.067431,0.475209,0.789540,108.0,66.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,9.763165,-4.163253,0,0.0
2,203.0,1558.081794,440.186028,19.898377,24.494291,5.832117,0.333387,0.504222,149.0,105.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,12.027867,-4.797089,0,0.0
3,202.0,750.921810,531.726206,1.183281,7.025863,6.972678,0.434480,1.143586,161.0,169.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,9.468642,-5.427308,0,0.0
4,201.0,424.333094,156.669725,16.976344,14.241267,6.072868,0.339266,0.712789,105.0,151.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211210,8.638670,-5.537292,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55959,2.0,14.176969,14.112522,5.592409,4.014829,4.165132,1.519961,2.022739,5.0,7.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-6.929020,2.694182,3,0.0
55960,2.0,17.252419,8.129460,5.073412,4.869400,3.681744,1.881460,1.838313,6.0,7.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-7.241755,2.280913,3,0.0
55961,2.0,19.016496,2.305615,8.628081,5.526589,5.818022,2.927572,1.447567,6.0,3.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-7.692024,3.093005,3,0.0
55962,2.0,29.013724,14.713282,4.932474,3.433771,3.853484,1.959405,1.111740,5.0,2.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,dmso,1s,211221,-6.790147,2.375580,3,0.0


# ck666

In [34]:
ada_ck666_ck666_group = {}
ada_ck666_ck666_group['path'] = path_drugs
ada_ck666_ck666_group['df'] = 'df_ada_ck666_ck666_merged_features'
ada_ck666_ck666_group['tracks'] = 'merged_ada_ck666_ck666_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_ck666_ck666'] = ada_ck666_ck666_group
np.save('analysis_metadata', analysis_metadata)

In [35]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_ck666_ck666',
                                         [1],
                                         ['Cell', 'ck666'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
31_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell1_1s
32_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell2_1s
33_211210_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell3_1s
37_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell1_1s
38_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell2_1s
39_211214_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell3_1s
49_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell1_1s
50_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell2_1s
51_211221_ap2-dnm2-arpc3_ap2-dnm2_drug_ck666_Cell3_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [36]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_ck666_ck666')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,216.0,498.789827,185.287239,2.827872,3.125548,4.542138,0.600730,0.824149,132.0,217.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.467757,-6.797040,0
1,213.0,524.079877,128.395344,8.612915,11.259526,5.074313,0.667881,0.963484,67.0,67.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.818717,-3.917402,0
2,208.0,787.298794,313.319461,12.868831,11.381641,6.142929,0.485429,0.784888,92.0,195.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,10.436287,-6.139174,0
3,206.0,962.220521,464.627450,12.038765,15.586400,3.488874,0.281071,0.643084,135.0,116.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,10.188956,-4.989708,0
4,205.0,704.303883,313.666548,6.972979,3.818180,4.485363,0.326309,0.827579,161.0,186.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.535883,-5.947090,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52679,2.0,26.702751,14.118350,3.575432,3.652531,2.684324,1.985302,1.151908,6.0,3.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-6.732839,2.214097,3
52680,2.0,38.495564,38.386164,0.632412,0.434158,4.211637,0.883096,0.399532,6.0,7.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,4.859496,4.264623,2
52681,2.0,20.387318,16.449435,0.959910,2.715558,7.660521,1.601248,2.258539,6.0,6.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-6.574395,2.225519,3
52682,2.0,31.654234,8.556023,1.513219,1.513219,3.120433,1.481886,0.137565,6.0,11.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-7.642763,0.732776,3


In [37]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_ck666_ck666')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,216.0,498.789827,185.287239,2.827872,3.125548,4.542138,0.600730,0.824149,132.0,217.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.467757,-6.797040,0,0.0
1,213.0,524.079877,128.395344,8.612915,11.259526,5.074313,0.667881,0.963484,67.0,67.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.818717,-3.917402,0,0.0
2,208.0,787.298794,313.319461,12.868831,11.381641,6.142929,0.485429,0.784888,92.0,195.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,10.436287,-6.139174,0,0.0
3,206.0,962.220521,464.627450,12.038765,15.586400,3.488874,0.281071,0.643084,135.0,116.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,10.188956,-4.989708,0,0.0
4,205.0,704.303883,313.666548,6.972979,3.818180,4.485363,0.326309,0.827579,161.0,186.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211210,9.535883,-5.947090,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52679,2.0,26.702751,14.118350,3.575432,3.652531,2.684324,1.985302,1.151908,6.0,3.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-6.732839,2.214097,3,0.0
52680,2.0,38.495564,38.386164,0.632412,0.434158,4.211637,0.883096,0.399532,6.0,7.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,4.859496,4.264623,2,0.0
52681,2.0,20.387318,16.449435,0.959910,2.715558,7.660521,1.601248,2.258539,6.0,6.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-6.574395,2.225519,3,0.0
52682,2.0,31.654234,8.556023,1.513219,1.513219,3.120433,1.481886,0.137565,6.0,11.0,...,ap2-dnm2-arpc3,ap2-dnm2,drug,ck666,1s,211221,-7.642763,0.732776,3,0.0


# hypotonic data, merged 211203 + 211208, before shock

In [38]:
path_hypotonic_merged='/Volumes/Google Drive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/hypotonic_tracking_data_merged_211203_211208_ap2dnm2/'

In [39]:
ada_hypotonic_before_group = {}
ada_hypotonic_before_group['path'] = path_hypotonic_merged
ada_hypotonic_before_group['df'] = 'df_ada_hypotonic_before_merged_features'
ada_hypotonic_before_group['tracks'] = 'merged_ada_hypotonic_before_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_hypotonic_before'] = ada_hypotonic_before_group
np.save('analysis_metadata', analysis_metadata)

In [40]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_hypotonic_before',
                                         [1],
                                         ['Cell', 'before'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
22_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell1_1s
24_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell2_1s
26_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell3_1s
28_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell4_1s
30_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell5_1s
62_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell1_1s
64_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell2_1s
66_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell3_1s
68_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell4_1s
70_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_before_Cell5_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [41]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_hypotonic_before')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,214.0,377.798001,207.837669,2.816601,6.237207,3.880280,0.442730,0.894743,168.0,149.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.226541,-4.388861,0
1,207.0,912.649800,226.664902,5.586988,4.275330,5.280566,0.668868,1.023994,161.0,104.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.165220,-4.239877,0
2,195.0,362.120149,335.352165,10.958896,10.719659,2.915594,0.334168,0.657747,99.0,142.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.533231,-5.132499,0
3,193.0,1100.749483,343.852558,6.511609,2.894950,6.365068,0.502067,0.928109,82.0,99.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.303862,-5.164870,0
4,190.0,880.934932,324.005804,6.101632,4.389072,5.028064,0.557094,0.770188,80.0,182.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.903669,-5.781659,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74287,2.0,21.977220,24.858384,1.391991,5.368103,4.945882,1.693168,2.468142,5.0,3.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,2.717681,5.123375,2
74288,2.0,26.732314,25.937549,5.681365,6.393079,3.624087,2.062259,2.332317,6.0,1.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,2.272141,4.703482,2
74289,2.0,71.479696,1.924836,4.423582,3.529405,5.625927,2.967790,1.593973,6.0,6.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,-7.255545,1.084829,3
74290,2.0,30.616907,3.472921,1.459682,0.524550,5.517637,2.014449,1.039189,6.0,1.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,-7.405970,2.562723,3


In [42]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_hypotonic_before')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,214.0,377.798001,207.837669,2.816601,6.237207,3.880280,0.442730,0.894743,168.0,149.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.226541,-4.388861,0,0.0
1,207.0,912.649800,226.664902,5.586988,4.275330,5.280566,0.668868,1.023994,161.0,104.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.165220,-4.239877,0,0.0
2,195.0,362.120149,335.352165,10.958896,10.719659,2.915594,0.334168,0.657747,99.0,142.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.533231,-5.132499,0,1.0
3,193.0,1100.749483,343.852558,6.511609,2.894950,6.365068,0.502067,0.928109,82.0,99.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.303862,-5.164870,0,0.0
4,190.0,880.934932,324.005804,6.101632,4.389072,5.028064,0.557094,0.770188,80.0,182.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211203,9.903669,-5.781659,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74287,2.0,21.977220,24.858384,1.391991,5.368103,4.945882,1.693168,2.468142,5.0,3.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,2.717681,5.123375,2,0.0
74288,2.0,26.732314,25.937549,5.681365,6.393079,3.624087,2.062259,2.332317,6.0,1.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,2.272141,4.703482,2,0.0
74289,2.0,71.479696,1.924836,4.423582,3.529405,5.625927,2.967790,1.593973,6.0,6.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,-7.255545,1.084829,3,0.0
74290,2.0,30.616907,3.472921,1.459682,0.524550,5.517637,2.014449,1.039189,6.0,1.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,before,1s,211208,-7.405970,2.562723,3,0.0


# hypotonic data, merged 211203 + 211208, after shock

In [43]:
ada_hypotonic_after_group = {}
ada_hypotonic_after_group['path'] = path_hypotonic_merged
ada_hypotonic_after_group['df'] = 'df_ada_hypotonic_after_merged_features'
ada_hypotonic_after_group['tracks'] = 'merged_ada_hypotonic_after_valid_tracks'
analysis_metadata.item()['experiment_groups']['ada_hypotonic_after'] = ada_hypotonic_after_group
np.save('analysis_metadata', analysis_metadata)

In [44]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'ada_hypotonic_after',
                                         [1],
                                         ['Cell', 'after'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
21_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell1_1s
23_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell2_1s
25_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell3_1s
27_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell4_1s
29_211203_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell5_1s
61_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell1_1s
63_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell2_1s
65_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell3_1s
67_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell4_1s
69_211208_ap2-dnm2-arpc3_ap2-dnm2_hypotonic_after_Cell5_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [45]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'ada_hypotonic_after')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,221.0,612.395723,790.107943,12.820705,11.244047,2.805439,0.291480,0.522397,105.0,160.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,11.973454,-5.191830,0
1,205.0,461.802189,477.128770,12.598194,10.328151,3.816827,0.426441,0.886652,145.0,189.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.726588,-5.510435,0
2,197.0,956.497498,516.720062,10.614360,14.461734,4.918462,0.267942,0.569825,152.0,157.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,10.273557,-5.804641,0
3,188.0,559.280718,438.074133,5.738161,8.987802,3.449380,0.438291,0.844328,179.0,181.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.137714,-5.476940,0
4,181.0,1172.670487,470.552115,4.965666,3.962355,4.151467,0.387121,0.707970,137.0,131.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.749118,-5.015876,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99127,2.0,28.004409,15.724082,5.586897,2.909973,5.419451,1.769276,1.757802,6.0,11.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,-7.313459,1.107294,3
99128,2.0,23.159565,78.080947,1.187148,1.714258,3.277268,0.639110,0.666563,5.0,7.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,6.635333,5.766887,2
99129,2.0,64.212530,43.139870,6.493910,1.697647,3.391303,2.064701,0.455061,5.0,5.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,3.088456,3.478826,2
99130,2.0,33.625944,15.269128,3.025869,2.125885,3.633861,2.170356,0.759286,5.0,11.0,...,2.0,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,-7.392783,0.770064,3


In [46]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'ada_hypotonic_after')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,221.0,612.395723,790.107943,12.820705,11.244047,2.805439,0.291480,0.522397,105.0,160.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,11.973454,-5.191830,0,0.0
1,205.0,461.802189,477.128770,12.598194,10.328151,3.816827,0.426441,0.886652,145.0,189.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.726588,-5.510435,0,0.0
2,197.0,956.497498,516.720062,10.614360,14.461734,4.918462,0.267942,0.569825,152.0,157.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,10.273557,-5.804641,0,0.0
3,188.0,559.280718,438.074133,5.738161,8.987802,3.449380,0.438291,0.844328,179.0,181.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.137714,-5.476940,0,0.0
4,181.0,1172.670487,470.552115,4.965666,3.962355,4.151467,0.387121,0.707970,137.0,131.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211203,9.749118,-5.015876,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99127,2.0,28.004409,15.724082,5.586897,2.909973,5.419451,1.769276,1.757802,6.0,11.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,-7.313459,1.107294,3,0.0
99128,2.0,23.159565,78.080947,1.187148,1.714258,3.277268,0.639110,0.666563,5.0,7.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,6.635333,5.766887,2,0.0
99129,2.0,64.212530,43.139870,6.493910,1.697647,3.391303,2.064701,0.455061,5.0,5.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,3.088456,3.478826,2,0.0
99130,2.0,33.625944,15.269128,3.025869,2.125885,3.633861,2.170356,0.759286,5.0,11.0,...,ap2-dnm2-arpc3,ap2-dnm2,hypotonic,after,1s,211208,-7.392783,0.770064,3,0.0
